In [3]:
from keras import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [16]:
# datagen = ImageDataGenerator(rescale=1.0 / 255.0, validation_split=0.2)
datagen = ImageDataGenerator(rescale=1.0 / 255.0,
                             rotation_range=20, shear_range=0.1,
                             zoom_range=0.2, channel_shift_range=0.1, vertical_flip=True, width_shift_range=0.1,
                             height_shift_range=0.1,
                             validation_split=0.1
                             )
target_size = 640
train_it = datagen.flow_from_directory('ground_truth_small/train', class_mode='binary', batch_size=16, target_size=(target_size, target_size), subset='training')
valid_it = datagen.flow_from_directory('ground_truth_small/train', class_mode='binary', batch_size=16, target_size=(target_size, target_size), subset='validation')
test_it = datagen.flow_from_directory('ground_truth_small/test', class_mode='binary', batch_size=16, target_size=(target_size, target_size))


Found 3090 images belonging to 2 classes.
Found 343 images belonging to 2 classes.
Found 389 images belonging to 2 classes.


In [17]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(target_size, target_size, 3)))
model.add(MaxPooling2D((10, 10)))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D((7, 7)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7024578642685740067
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5738856448
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18275236879838524399
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [18]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 636, 636, 32)      2432      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 61, 61, 16)        4624      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 8, 8, 16)         0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 1024)              0         
                                                                 
 dense_4 (Dense)             (None, 128)              

In [19]:
model.fit(train_it, validation_data=valid_it, epochs=40)

Epoch 1/40
194/194 [==============================] - 209s 1s/step - loss: 0.3246 - accuracy: 0.8586 - precision_2: 0.8452 - recall_2: 0.9182 - val_loss: 1.0151 - val_accuracy: 0.5452 - val_precision_2: 1.0000 - val_recall_2: 0.1959
Epoch 2/40
194/194 [==============================] - 209s 1s/step - loss: 0.1998 - accuracy: 0.9207 - precision_2: 0.9336 - recall_2: 0.9256 - val_loss: 0.6868 - val_accuracy: 0.6939 - val_precision_2: 0.9785 - val_recall_2: 0.4691
Epoch 3/40
194/194 [==============================] - 206s 1s/step - loss: 0.1667 - accuracy: 0.9353 - precision_2: 0.9484 - recall_2: 0.9365 - val_loss: 0.7150 - val_accuracy: 0.7114 - val_precision_2: 0.9798 - val_recall_2: 0.5000
Epoch 4/40
194/194 [==============================] - 208s 1s/step - loss: 0.1492 - accuracy: 0.9401 - precision_2: 0.9520 - recall_2: 0.9416 - val_loss: 0.6992 - val_accuracy: 0.6735 - val_precision_2: 0.9767 - val_recall_2: 0.4330
Epoch 5/40
194/194 [==============================] - 207s 1s/step -

In [20]:
model.save('models/clasiffication_custom_640.h5')

In [21]:
model.evaluate(test_it)

25/25 [==============================] - 26s 1s/step - loss: 0.1659 - accuracy: 0.9537 - precision_2: 0.9780 - recall_2: 0.9271


[0.16594529151916504,
 0.9537274837493896,
 0.9780219793319702,
 0.9270833134651184]